In [88]:
#Pacotes que estamos importando
import numpy as np               
import pandas as pd             
from scipy.stats import rankdata # Para o ranking das candidatas

In [108]:
# Dados

attributes = np.array(["Capacidade", "Autonomia", "Eficiência", "Preço"])
candidates = np.array(["DJ", "Pelicano", "Dractor"])
raw_data = np.array([
    [40, 102,  21.3,  229999],
    [30, 68,  9.3,  170000],
    [10, 240,  6,   159999],
    
])

#print(raw_data)

weights = np.array([0.125, 0.125, 0.25, 0.5])

# The indices of the attributes (zero-based) that are considered beneficial.
# Those indices not mentioned are assumed to be cost attributes.
benefit_attributes = set([0,1,2])

# Display the raw data we have
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,Capacidade,Autonomia,Eficiência,Preço
DJ,40.0,102.0,21.3,229999.0
Pelicano,30.0,68.0,9.3,170000.0
Dractor,10.0,240.0,6.0,159999.0


In [109]:
#Step 1
#Normalização

m = len(raw_data)
n = len(attributes)
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.max(column)

raw_data /= divisors

columns = ["$C_{%d}$" % j for j in range(n)]
pd.DataFrame(data=raw_data, index=candidates, columns=columns)


,$C_{0}$,$C_{1}$,$C_{2}$,$C_{3}$
DJ,1.00,0.425000,1.00000,1.000000
Pelicano,0.75,0.283333,0.43662,0.739134
Dractor,0.25,1.000000,0.28169,0.695651


In [117]:
#Step 2

# Criando uma matriz com todas as colunas
all_columns = np.zeros((raw_data.shape[0], len(attributes)))

# Preenchendo as colunas de benefício com os valores correspondentes
all_columns[:, list(benefit_attributes)] = raw_data[:, list(benefit_attributes)]

print(all_columns)

benefit_profit = all_columns*weights

print(benefit_profit)
cost_profit = (raw_data - all_columns)*weights

#Pra não atrapalhar, vou tirar as colunas que não são zeros 

# Verificar se todos os elementos de cada coluna são iguais a zero
nonzero_columns = np.all(benefit_profit != 0, axis=0) 
benefit_profit = benefit_profit[:, nonzero_columns] # Remover as colunas cujos elementos são todos iguais a zero

print(benefit_profit)

# Verificar se todos os elementos de cada coluna são iguais a zero
nonzero_columns = np.all(cost_profit != 0, axis=0) 
cost_profit = cost_profit[:, nonzero_columns] # Remover as colunas cujos elementos são todos iguais a zero

column_sums = np.sum(benefit_profit, axis=1)
column_sums2 =np.sum(cost_profit, axis=1)
column_products = np.prod(benefit_profit, axis=1)
column_products2 = np.prod(cost_profit, axis=1)

#print(cost_profit)

u_i_wsd = column_sums - column_sums2
u_i_wpd = column_products- column_products2
u_i_wsr = column_sums/column_sums2
u_i_wpr = column_products/column_products2

#print(benefit_profit)
#print(row_sums)
#print(row_sums2)
#print(benefit_profit)
#print(cost_profit)
#print(column_sums)
#print(column_sums2)

#print(column_products)
#print(column_products2)
#print(column_products)
#print(column_products2)

#print(u_i_wsd)
#print(u_i_wpd)
#print(u_i_wsr)
#print(u_i_wpr)

df = pd.DataFrame({
    'u_i_wsd': u_i_wsd,
    'u_i_wpd': u_i_wpd,
    'u_i_wsr': u_i_wsr,
    'u_i_wpr': u_i_wpr
},index=candidates)

# Exibir o DataFrame
print(df)

[[1.         0.425      1.         0.        ]
 [0.75       0.28333333 0.43661972 0.        ]
 [0.25       1.         0.28169014 0.        ]]
[[0.125      0.053125   0.25       0.        ]
 [0.09375    0.03541667 0.10915493 0.        ]
 [0.03125    0.125      0.07042254 0.        ]]
[[0.125      0.053125   0.25      ]
 [0.09375    0.03541667 0.10915493]
 [0.03125    0.125      0.07042254]]
           u_i_wsd   u_i_wpd   u_i_wsr   u_i_wpr
DJ       -0.071875 -0.498340  0.856250  0.003320
Pelicano -0.131245 -0.369204  0.644867  0.000981
Dractor  -0.121153 -0.347550  0.651685  0.000791


In [113]:
#Step 3

u_i_wsd2 = u_i_wsd/(1+np.max(u_i_wsd))
u_i_wpd2 = u_i_wpd/(1+np.max(u_i_wpd))
u_i_wsr2 = u_i_wsr/(1+np.max(u_i_wsr))
u_i_wpr2 = u_i_wpr/(1+np.max(u_i_wpr))

df = pd.DataFrame({
    'u_i_wsd2': u_i_wsd2,
    'u_i_wpd2': u_i_wpd2,
    'u_i_wsr2': u_i_wsr2,
    'u_i_wpr2': u_i_wpr2
}, index = candidates)

# Exibir o DataFrame
print(df)



          u_i_wsd2  u_i_wpd2  u_i_wsr2  u_i_wpr2
DJ        0.587629 -1.352941  0.708029  0.298246
Pelicano  0.301369 -1.520828  0.580657  0.088090
Dractor   0.344758 -1.471125  0.642864  0.071040


In [112]:
column_sums = df.sum(axis=1)/4
print(column_sums)


DJ          0.060241
Pelicano   -0.137678
Dractor    -0.103116
dtype: float64
